# Sample accessioning

This example shows the Ovation Service Lab (OSL) APIs for sample accessioning by creating a simple `Requisition` with one sample.

## Setup

In [10]:
import uuid

from pprint import pprint
from datetime import date

from ovation.session import connect

## Session

In [2]:
s = connect(input("Email: "), api='https://services-staging.ovation.io', token='/api/v1/sessions')

Email: barry@ovation.io


## Parameters

Many OSL APIs require the Organization `id`.

In [3]:
organization_id = input('Organization id: ')

Organization id: 156


## Requisition

Create a `container` for the sample (in this case, a Tube). The tube `identifier` and `barcode` will be generated by Ovation. You can supply them using the `identifier` and `barcode` attributes of the `container` if needed:

In [14]:
tube = s.post(s.entity_path('container'), 
              data={'container': {'type': 'Tube'}},
              params={'organization_id': organization_id})

We need to know which project this Requisition belongs to. You can find projects by ID or `name`:

In [11]:
project = s.get(s.entity_path('project'), 
                params={'q': 'NU0001SC-LS', # Find project by name
                        'organization_id': organization_id}).projects[0]
pprint(project)

{'archived': False,
 'created_at': '2016-10-05T22:36:19.000Z',
 'custom_attributes': {},
 'id': 227,
 'is_admin': True,
 'links': {'documents': '/api/v1/documents?project_id=227',
           'folders': '/api/v1/projects/227/links/folders',
           'memberships': '/api/v1/projects/227/links/memberships',
           'tags': '/api/v1/projects/227/links/tags',
           'training_packs': '/api/v1/projects/227/links/training_packs'},
 'name': 'NU0001SC-LS',
 'queued_job_ids': [],
 'research_project_url': None,
 'team': {'id': 2587,
          'membership_ids': [4025, 3968, 3969, 3970, 3967, 3971],
          'name': 'NU0001SC-LS',
          'organization_id': None,
          'pending_membership_ids': [],
          'project_id': 227,
          'role_ids': [],
          'uuid': 'e706ec83-b586-463b-ba89-fca53dd49051'},
 'uuid': 'f6270330-d226-400e-8a90-fd2d10f81071'}


Create a `Requisition` and the `Sample`:

In [15]:
requisition_data = {"identifier": str(uuid.uuid4()), # Any unique (within organization) identifier
                    "template": "RNA Requisition", # The requisition template, for the selected project
                    "project_id": project.id,
                    "organization_id": organization_id,
                    "custom_attributes": {'my-attribute': 1.0}, # Optional; Requisition custom attributes
                    "samples": [
                        {"identifier": str(uuid.uuid4()), # Any unique (within organization) identifier
                         "date_received": date.today().isoformat(),
                         "custom_attributes": {'my-sample-attribute': 1.0}, # Optional; Sample custom attributes
                         "sample_states": [
                                {"container_id": tube.id,
                                "position": "A01"}
                            ]
                        }
                    ]
                  }

req = s.post(s.entity_path('requisition'), 
             data={'requisition': requisition_data},
             params={'organization_id': organization_id})

pprint(req)

{'requisition': {'barcode_link': 'https://services-staging.ovation.io/api/v1/requisitions/2256.obc',
                 'billing_information': None,
                 'complete': False,
                 'contact_email': None,
                 'created_at': 1477370554,
                 'custom_attributes': {'my-attribute': 1.0},
                 'date_received': '2016-10-25 00:00:00',
                 'diagnosis': None,
                 'has_results': False,
                 'id': 2256,
                 'identifier': '6cd4fb6a-2cff-476f-b122-857b8b27c61a',
                 'is_admin': True,
                 'links': {'documents': '/api/v1/documents?requisition_id=2256',
                           'folders': '/api/v1/requisitions/2256/links/folders',
                           'memberships': '/api/v1/requisitions/2256/links/memberships',
                           'tags': '/api/v1/requisitions/2256/links/tags'},
                 'medications': None,
                 'medications_pt': None,


In [8]:
requisition_data

{'custom_attributes': {'my-attribute': 1.0},
 'identifier': 'c71f9e37-d0cf-4e96-9b48-79fb44d87f53',
 'organization_id': '156',
 'project_id': 227,
 'samples': [{'custom_attributes': {'my-sample-attribute': 1.0},
   'date_received': '2016-10-24',
   'identifier': 'd863436d-6a38-40ec-ad1c-40d4aec42142',
   'sample_states': [{'container_id': 4906, 'position': 'A01'}]}],
 'template': 'Basic'}